In [ ]:
## Prediction

%reset -f

import numpy as np
import scipy as sp
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.models import Model
from keras.layers.merge import concatenate
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.layers import Dense, Input, Lambda, Dot
from tensorflow.keras.layers.experimental import preprocessing

with tf.device('/device:cpu:0'):
    
    dof = 76
    dt = 0.01
    min_t = 0
    max_t = 2

    t = np.arange(min_t, max_t, dt)
    lt = t.shape[0]
    
    predicted = np.zeros([10000,200])
    x_test_0_new = np.zeros([2000000, 100])
    x_test_1_new = np.zeros([2000000, 1])
        
    string = 'data_10000_samples_fs_100Hz_76_storey_NL_20FT.mat'
    data = spi.loadmat(string)
    
    f = data['f']
    disp = data['y'][0:dof,:,:]
    
    test_samples = 10000
    x_test = f[0:-1:2,:].T

    for disp_num in range(0,76):
        y = disp[disp_num,:,:].T

        string = 'model_save_76DOF_NONLINEAR_400_DISP_'+str(disp_num+1)
        model = tf.keras.models.load_model(string)
        print('5DOF_nonlinear_400_displacement_'+str(disp_num+1))

        for i in range(0,test_samples):
            x_test_0_new[lt*i:lt+lt*i,:] = np.tile(x_test[i,:],[200,1])
            x_test_1_new[lt*i:lt+lt*i,:] = t.reshape([lt,1])

        predictions_new = model({"input_a":x_test_0_new, "input_b":x_test_1_new}).numpy()

        for i in range(0,test_samples):
            predicted[i,:] = predictions_new[lt*i:lt+lt*i].reshape(1,-1)

#         string = '76DOF_training_samples_400_displacement_'+str(disp_num+1)+'_FT_20.mat' 
#         spi.savemat(string,{'actual_y': y, 'predicted_y': predicted, 'model_num': 400,
#                             'disp_num': disp_num+1, 'fourier_terms': 20})